# Defect Pills Classification
This Notebook contains CNN which trained for Defect Pills Classification.
Dataset contains 215 defect pills and 140 normal pills. It can be found on the following website: https://www.kaggle.com/pudpawat/pill-defect-dataset

If you own this dataset and want my notebook taken down, please contact me and I will do that immediately.

In [1]:
# Import modules
import tensorflow as tf
from tensorflow.keras import layers
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
# Create data folder variables
data_dir = 'Pill-Defects-main/'
defect_folder = data_dir+'defect'
normal_folder = data_dir+'normal'

# List with image names
defect_names=os.listdir(defect_folder)
normal_names=os.listdir(normal_folder)

# Number of defect and normal images
defect_count = len(defect_names)
normal_count = len(normal_names)

In [3]:
# Size of each image
w, h = cv2.imread('/'.join([defect_folder, defect_names[0]]), cv2.IMREAD_GRAYSCALE).shape

total_img = defect_count+normal_count # Total number of images

# Create input array
X = np.zeros((total_img, w, h), dtype=np.float)

for i in range(defect_count):
    X[i, :, :] = cv2.imread('/'.join([defect_folder, defect_names[i]]), cv2.IMREAD_GRAYSCALE).astype(np.float)/255

for i in range(normal_count):
    X[i+defect_count, :, :] = cv2.imread('/'.join([normal_folder, normal_names[i]]), cv2.IMREAD_GRAYSCALE).astype(np.float)/255

# Create output array
Y = np.hstack((np.zeros(defect_count), np.ones(normal_count)))

In [4]:
# Divide dataset into train/test sets
np.random.seed()
ind = np.arange(total_img)
ind = np.random.permutation(ind)
train_ind = int(0.8*total_img)

X_train = X[ind[0:train_ind], :, :]
X_test = X[ind[train_ind+1:], :, :]
Y_train = Y[ind[0:train_ind]]
Y_test = Y[ind[train_ind+1:]]

# Expand dims to create suitable input array
X_train = np.expand_dims(X_train, 3)
X_test = np.expand_dims(X_test, 3)

In [5]:
# Build CNN model
model = tf.keras.models.Sequential(
    [
        layers.Conv2D(8, 3, padding='same', activation='relu', input_shape=(512,512, 1)),
        layers.MaxPool2D(pool_size=(2,2), strides=None),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPool2D(pool_size=(2,2), strides=None),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPool2D(pool_size=(2,2), strides=None),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPool2D(pool_size=(2,2), strides=None),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPool2D(pool_size=(2,2), strides=None),
        layers.Conv1D(16, 3, padding='same', activation='relu'), # Reduce number of filters
        layers.Flatten(), # Prepare data for Dense layers
        layers.Dense(32, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid') # Sigmoid is used for Binary Classification problems
    ]
)

# Return summary of created CNN
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 512, 512, 8)       80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 256, 256, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 16)      1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        1

In [6]:
# Compile model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [7]:
# Train model
model.fit(X_train, Y_train, batch_size=64, epochs=20, validation_split=0.2, callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5))

Epoch 1/20
4/4 [==============================] - 9s 2s/step - loss: 0.6847 - accuracy: 0.6132 - val_loss: 0.6787 - val_accuracy: 0.5789
Epoch 2/20
4/4 [==============================] - 8s 2s/step - loss: 0.6619 - accuracy: 0.6069 - val_loss: 0.6638 - val_accuracy: 0.5789
Epoch 3/20
4/4 [==============================] - 8s 2s/step - loss: 0.6508 - accuracy: 0.5975 - val_loss: 0.6473 - val_accuracy: 0.5789
Epoch 4/20
4/4 [==============================] - 8s 2s/step - loss: 0.6208 - accuracy: 0.6074 - val_loss: 0.6313 - val_accuracy: 0.5789
Epoch 5/20
4/4 [==============================] - 8s 2s/step - loss: 0.5781 - accuracy: 0.6199 - val_loss: 0.5680 - val_accuracy: 0.5789
Epoch 6/20
4/4 [==============================] - 8s 2s/step - loss: 0.5481 - accuracy: 0.6033 - val_loss: 0.4808 - val_accuracy: 0.5789
Epoch 7/20
4/4 [==============================] - 8s 2s/step - loss: 0.5014 - accuracy: 0.6937 - val_loss: 0.5673 - val_accuracy: 0.7544
Epoch 8/20
4/4 [=========================

In [8]:
# Test model
test_scores = model.evaluate(X_test, Y_test, verbose=2)
print("Test loss: ", test_scores[0])
print("Test accuracy: ", test_scores[1])

3/3 - 0s - loss: 8.9981e-06 - accuracy: 1.0000
Test loss:  8.998094017442781e-06
Test accuracy:  1.0


In [10]:
# Save model
model.save_weights('weights/final_weights')
model.save('models/defect_pill_detection_model')

INFO:tensorflow:Assets written to: models/defect_pill_detection_model\assets


# Results
Results show that accuracy is 100% on both train/val and test sets. This shows that CNNs are a great approach to solving this type of problems.